# Genetic Algorithms

## Chapter 5 

Depend on stochastic processes of selection, crossover, and mutation. Genetic algorithms are not generally used in software engineering, but can be a useful method for solving otherwise intractable problems.

# Chromosome

* Determine own fitness
* Create instance with randomly selected genes - first generation
* Implement crossover (combine with same type to create children) - mix with another chromosome
* Mutute - small, randomly change in itself

Codified in code following.

In [1]:
from __future__ import annotations
from typing import TypeVar, Tuple, Type
from abc import ABC, abstractmethod

# type T is bound to chromosome, so any type declared as T must be chromosome
# or subclass of chromosome
T = TypeVar("T", bound="chromosome")

In [2]:
class Chromosome(ABC):
    """
    Base class for all chromosomes. fitness, random_instance,
    crossover, and mutate must be overwritten."""

    @abstractmethod
    def fitness(self) -> float:
        ...

    @classmethod
    @abstractmethod
    def random_instance(cls: Type[T]) -> T:
        ...

    @abstractmethod
    def crossover(self: T, other: T) -> Tuple[T, T]:
        ...

    @abstractmethod
    def mutate(self) -> None:
        ...

## Genetic Algorithm Procedure

1. Initial population of random chromosomes for first generation.
2. Mesure fitness of each chromosome in generation. If any exceeds threshold, return it and end algorithm.
3. Select individuals to reproduce with higher probability of selecting those with higher fitness (roulette and tournamen selection).
4. Crossover - combine - some of the selected chromosomes, with a probability to create children that represent next genertion.
5. Mutute - with a low probability some of the chromosomes. Population of new generation is done and replaces old population (previous generation).
6. Return to step 2 unless max number of generations reached. In case of max number reached, return best chromosome thus far.

Many individual questions to answer that determine the algorithm. These form the hyperparameters controlling a genetic algorithm.

### Selection Methods

* Tournament selection: pit chromosomes against one another and choose winner based on fitness
* Roulette Selection: random selection of chromosomes with probabilities proportional to fitness

In [32]:
from __future__ import annotations
from typing import TypeVar, Generic, List, Tuple, Callable
from enum import Enum
from random import choices, random
from heapq import nlargest
from statistics import mean

C = TypeVar("C", bound=Chromosome)


class GeneticAlgorithm(Generic[C]):
    SelectionType = Enum("SelectionType", "ROULETTE TOURNAMENT")

    def __init__(
        self,
        initial_population: List[C],
        threshold: float,
        max_generations: int = 100,
        mutation_chance: float = 0.01,
        crossover_chance: float = 0.7,
        # Tournament pits chromosomes against one another
        selection_type: SelectionType = SelectionType.TOURNAMENT,
    ) -> None:
        # First population is randomly initialized using the chromosomes random_instance() method
        # Could be better selected: known as seeding
        self._population: List[C] = initial_population
        self._threshold: float = threshold
        self._max_generations: int = max_generations
        self._mutation_chance: float = mutation_chance
        self._crossover_chance: float = crossover_chance
        self._selection_type: GeneticAlgorithm.SelectionType = selection_type
        self._fitness_key: Callable = type(self._population[0]).fitness

    # Use the probability distribution wheel to pick 2 parents
    # Note: will not work with negative fitness results
    def _pick_roulette(self, wheel: List[float]) -> Tuple[C, C]:
        # Select from list with probabilities
        return tuple(choices(self._population, weights=wheel, k=2))

    # Choose num_participants at random and take the best 2 for parents
    def _pick_tournament(self, num_participants: int) -> Tuple[C, C]:
        participants: List[C] = choices(self._population, k=num_participants)
        # Pick two chromosomes with highest fitness
        return tuple(nlargest(2, participants, key=self._fitness_key))

    # Replace the population with a new generation of individuals
    def _reproduce_and_replace(self) -> None:
        new_population: List[C] = []
        # keep going until we've filled the new generation
        while len(new_population) < len(self._population):
            # pick the 2 parents
            if self._selection_type == GeneticAlgorithm.SelectionType.ROULETTE:
                parents: Tuple[C, C] = self._pick_roulette(
                    [x.fitness() for x in self._population]
                )
            else:
                # Run tournament with n/2 participants where n is the current size
                # of the population
                parents = self._pick_tournament(len(self._population) // 2)
            # potentially crossover the 2 parents
            if random() < self._crossover_chance:
                new_population.extend(parents[0].crossover(parents[1]))
            else:
                new_population.extend(parents)
        # if we had an odd number, we'll have 1 extra, so we remove it
        if len(new_population) > len(self._population):
            new_population.pop()
        self._population = new_population  # replace reference

    # With _mutation_chance probability mutate each individual
    def _mutate(self) -> None:
        for individual in self._population:
            # Use the individuals mutate method
            if random() < self._mutation_chance:
                individual.mutate()

    # Run the genetic algorithm for max_generations iterations
    # and return the best individual found
    def run(self, progress=100) -> C:
        # Keep track of how many times we found a better individual
        better_count: int = 0
        best: C = max(self._population, key=self._fitness_key)
        for generation in range(self._max_generations):
            # early exit if we beat threshold
            if best.fitness() >= self._threshold:
                print(
                    f"""Found best in {generation} generations. Best = {best.fitness()}. 
Found better individual in {100*(better_count/generation):.2f}% of generations."""
                )
                return best
            # Print progress every 1/10 max_generations
            if (generation + 1) % (self._max_generations // 10) == 0:
                print(
                    f"Generation {generation} Best {best.fitness()} Avg {mean(map(self._fitness_key, self._population))}"
                )
            # Create the next generation
            self._reproduce_and_replace()
            # Mutate individuals in the next generation
            self._mutate()
            highest: C = max(self._population, key=self._fitness_key)
            if highest.fitness() > best.fitness():
                best = highest  # found a new best
                better_count += 1
        return best  # best we found in _max_generations

Selection occurs during reproduction. The selection method determines two new parents for the next generation. 

`_mutate()` and `crossover` are left up to the individual chromosomes to determine. The individual chromosomes also determine `_fitness_key()`, the method by which the fitness of a chromosome is measured.

## Test of Genetic Algorithm Basic Class

Attempt to solve a quadratic equation using a genetic algorithm


$$6*x - x^2 + 4*y - y^2$$

Solving analytically:

$$6 - 2*x + 4 -2y = 0$$

$x=3$ and $y=2$ yields a maximum value of 13.

In [33]:
from __future__ import annotations
from typing import Tuple, List
from random import randrange, random
from copy import deepcopy

class SimpleEquation(Chromosome):
    def __init__(self, x: int, y: int) -> None:
        self.x: int = x
        self.y: int = y

    def fitness(self) -> float: # 6x - x^2 + 4y - y^2
        return 6 * self.x - self.x * self.x + 4 * self.y - self.y * self.y
    
    # Class method does not require creating an instance to run
    @classmethod
    def random_instance(cls) -> SimpleEquation:
        # Random initialization method
        return SimpleEquation(randrange(100), randrange(100))

    def crossover(self, other: SimpleEquation) -> Tuple[SimpleEquation, SimpleEquation]:
        # For crossover, swap the ys
        child1: SimpleEquation = deepcopy(self)
        child2: SimpleEquation = deepcopy(other)
        child1.y = other.y
        child2.y = self.y
        return child1, child2

    def mutate(self) -> None:
        # For mutation, either increase or decrease x/y
        if random() > 0.5: # mutate x
            if random() > 0.5:
                self.x += 1
            else:
                self.x -= 1
        else: # otherwise mutate y
            if random() > 0.5:
                self.y += 1
            else:
                self.y -= 1
    
    def __str__(self) -> str:
        # Simple representation of the simple equation
        return f"X: {self.x} Y: {self.y} Fitness: {self.fitness()}"

In [34]:
eq = SimpleEquation(2, 4)
print(eq)

X: 2 Y: 4 Fitness: 8


In [35]:
eq_2 = SimpleEquation(5, 8)
eq, eq_2 = eq.crossover(eq_2)
print(eq)

X: 2 Y: 8 Fitness: -24


In [36]:
for _ in range(10):
    eq.mutate()
    print(eq)

X: 3 Y: 8 Fitness: -23
X: 3 Y: 7 Fitness: -12
X: 2 Y: 7 Fitness: -13
X: 3 Y: 7 Fitness: -12
X: 2 Y: 7 Fitness: -13
X: 3 Y: 7 Fitness: -12
X: 4 Y: 7 Fitness: -13
X: 3 Y: 7 Fitness: -12
X: 2 Y: 7 Fitness: -13
X: 2 Y: 8 Fitness: -24


In [40]:
eqs = [SimpleEquation.random_instance() for _ in range(100)]
ga = GeneticAlgorithm(initial_population=eqs, threshold=13, max_generations=5000, mutation_chance=0.25, 
                      crossover_chance=0.5, selection_type='TOURNAMENT')

### Run the Genetic Algorithm to Maximize the Function

In [41]:
r = ga.run()

Found best in 8 generations. Best = 13. 
Found better individual in 87.50% of generations.


In [42]:
print(r)

X: 3 Y: 2 Fitness: 13


In a real-world situation, we would just use the analytical method to find the answer. The genetic algorithm takes more computational power and is not guaranteed to find the optimal answer given a limited number of iterations.

## Solving Cryptoarithmetic Puzzle Revisited

SEND + MORE = MONEY

where each letter can be represented by only one number.

I'll alter the book's implementation by preventing the use of 0 as one of the numbers.

In [68]:
from __future__ import annotations
from typing import Tuple, List
from random import shuffle, sample
from copy import deepcopy


class SendMoreMoney2(Chromosome):
    def __init__(self, letters: List[str]) -> None:
        self.letters: List[str] = letters

    def fitness(self) -> float:
        # Determine the "goodness" of a solutin
        s: int = self.letters.index("S")
        e: int = self.letters.index("E")
        n: int = self.letters.index("N")
        d: int = self.letters.index("D")
        m: int = self.letters.index("M")
        o: int = self.letters.index("O")
        r: int = self.letters.index("R")
        y: int = self.letters.index("Y")
        send: int = s * 1000 + e * 100 + n * 10 + d
        more: int = m * 1000 + o * 100 + r * 10 + e
        money: int = m * 10000 + o * 1000 + n * 100 + e * 10 + y
        difference: int = abs(money - (send + more))
        return 1 / (difference + 1)

    # Class method does not require creating an instance to use
    @classmethod
    def random_instance(cls) -> SendMoreMoney2:
        letters = ["S", "E", "N", "D", "M", "O", "R", "Y", ""]
        shuffle(letters)
        return SendMoreMoney2(letters)

    def crossover(self, other: SendMoreMoney2) -> Tuple[SendMoreMoney2, SendMoreMoney2]:
        """
        Mix two children to create two new children. 
        """
        child1: SendMoreMoney2 = deepcopy(self)
        child2: SendMoreMoney2 = deepcopy(other)
        # Sample 2 random indices for mixing
        idx1, idx2 = sample(range(len(self.letters)), k=2)
        # Select the letters
        l1, l2 = child1.letters[idx1], child2.letters[idx2]
        # Swap the letters
        child1.letters[child1.letters.index(l2)] = child1.letters[idx2]
        child1.letters[idx2] = l2
        # Swap the letters
        child2.letters[child2.letters.index(l1)] = child2.letters[idx1]
        child2.letters[idx1] = l1
        return child1, child2

    def mutate(self) -> None:  # swap two letters' locations
        """
        Mutate an individual by swapping two of its letters. This could be adjusted to
        swap more letters at once.
        """
        idx1, idx2 = sample(range(len(self.letters)), k=2)
        self.letters[idx1], self.letters[idx2] = self.letters[idx2], self.letters[idx1]

    def __str__(self) -> str:
        s: int = self.letters.index("S")
        e: int = self.letters.index("E")
        n: int = self.letters.index("N")
        d: int = self.letters.index("D")
        m: int = self.letters.index("M")
        o: int = self.letters.index("O")
        r: int = self.letters.index("R")
        y: int = self.letters.index("Y")
        send: int = s * 1000 + e * 100 + n * 10 + d
        more: int = m * 1000 + o * 100 + r * 10 + e
        money: int = m * 10000 + o * 1000 + n * 100 + e * 10 + y
        difference: int = abs(money - (send + more))
        return f"{send} + {more} = {money} Difference: {difference}"

In [69]:
import copy

letters = ["S", "E", "N", "D", "M", "O", "R", "Y", " "]
one = SendMoreMoney2(copy.deepcopy(letters))

shuffle(letters)
two = SendMoreMoney2(copy.deepcopy(letters))

print(one)
print(two)

123 + 4561 = 45217 Difference: 40533
6043 + 7810 = 78405 Difference: 64552


In [70]:
new_one, new_two = one.crossover(two)
print(new_one)
print(new_two)

6123 + 4501 = 45217 Difference: 34593
6043 + 7210 = 72405 Difference: 59152


In [71]:
print(one.letters)
print(new_one.letters)
print(new_two.letters)

['S', 'E', 'N', 'D', 'M', 'O', 'R', 'Y', ' ']
['R', 'E', 'N', 'D', 'M', 'O', 'S', 'Y', ' ']
['E', 'R', 'O', 'D', 'N', 'Y', 'S', 'M', ' ']


The fitness function returns $$\frac{1}{\text{difference}+1}$$ because we want a metric to maximize. This means that the fitness will be 1 when the difference is 0.

## Solve Puzzle

As we have limited the algorithm to never use 0 as a value, this may not finish. The initial states are randomized.

In [72]:
initial = [SendMoreMoney2.random_instance() for _ in range(1000)]

ga = GeneticAlgorithm(
    initial_population=initial,
    threshold=1.0,
    max_generations=1000,
    mutation_chance=0.5,
    crossover_chance=0.5,
    selection_type="ROULETTE",
)
result = ga.run()

Found best in 87 generations. Best = 1.0. 
Found better individual in 12.64% of generations.


In [73]:
print(result)

7531 + 825 = 8356 Difference: 0


# Determining Most Efficient Ordering for Compressing A String

Ordering of a list will affect the compressed size. We can use a genetic algorithm to try and find the optimal ordering for the smallest compression.

In [74]:
import random_word

words = random_word.RandomWords().get_random_words()
len(words)

50

In [113]:
from random import shuffle, sample
from copy import deepcopy
from zlib import compress
from sys import getsizeof

from pickle import dumps

TARGET = deepcopy(words)
print(f'Initial size of compressed TARGET = {getsizeof(compress(dumps(words)))} bytes')

Initial size of compressed TARGET = 548 bytes


In [116]:
class ListCompression(Chromosome):
    def __init__(self, lst: List[Any]) -> None:
        self.lst: List[Any] = lst

    @property
    def bytes_compressed(self) -> int:
        return getsizeof(compress(dumps(self.lst)))

    def fitness(self) -> float:
        return 1 / self.bytes_compressed

    @classmethod
    def random_instance(cls) -> ListCompression:
        """
        Random initialization of list ordering.
        """
        mylst: List[str] = deepcopy(TARGET)
        shuffle(mylst)
        return ListCompression(mylst)

    def crossover(
        self, other: ListCompression
    ) -> Tuple[ListCompression, ListCompression]:
        """
        Crossover one item from first list with an item from the second list.
        """
        child1: ListCompression = deepcopy(self)
        child2: ListCompression = deepcopy(other)

        idx1, idx2 = sample(range(len(self.lst)), k=2)
        word1, word2 = child1.lst[idx1], child2.lst[idx2]

        # Swap the words
        child1.lst[idx2] = word2
        child1.lst[child1.lst.index(word2)] = child1.lst[idx2]

        # Swap the words
        child2.lst[idx1] = word1
        child2.lst[child2.lst.index(word1)] = child2.lst[idx1]
        return child1, child2

    def mutate(self) -> None:
        """
        Swap two locations within one chromosome.
        """
        idx1, idx2 = sample(range(len(self.lst)), k=2)
        self.lst[idx1], self.lst[idx2] = self.lst[idx2], self.lst[idx1]

    def __str__(self) -> str:
        return f"Order: {self.lst} Bytes: {self.bytes_compressed}."

### Run the Genetic Algorithm

Try and find the list ordering resulting in the smallest compressed list size.

In [117]:
initial = [ListCompression.random_instance() for _ in range(500)]
ga = GeneticAlgorithm(
    initial_population=initial,
    threshold=1.0,
    max_generations=1000,
    mutation_chance=0.25,
    crossover_chance=0.75,
    selection_type="TOURNAMENT",
)
result = ga.run()

Generation 99 Best 0.004149377593360996 Avg 0.004129142972397066
Generation 199 Best 0.004830917874396135 Avg 0.004798576075216512
Generation 299 Best 0.004830917874396135 Avg 0.004806546815553076
Generation 399 Best 0.004830917874396135 Avg 0.004795746999158475
Generation 499 Best 0.004830917874396135 Avg 0.0047974571497858805
Generation 599 Best 0.004830917874396135 Avg 0.004799389533887875
Generation 699 Best 0.004830917874396135 Avg 0.004798036904713716
Generation 799 Best 0.004830917874396135 Avg 0.00479105949152754
Generation 899 Best 0.004830917874396135 Avg 0.004799855139214671
Generation 999 Best 0.004830917874396135 Avg 0.004799433037271091


In [118]:
print(result)

Order: ['blurring', 'beamy', 'yashmaks', 'lengthed', 'shkotzim', 'lengthed', 'shkotzim', 'yashmaks', 'satays', 'diploidy', 'beamy', 'Lawrie', 'beamy', 'beamy', 'beginnings', 'pot-hunter', 'lengthed', 'shkotzim', 'Lawrie', 'beamy', 'beamy', 'shkotzim', 'Lawrie', 'rumseller', 'beamy', 'beamy', 'battleground', 'Lawrie', 'beamy', 'beginnings', 'blurring', 'lengthed', 'shkotzim', 'beginnings', 'blurring', 'beamy', 'shkotzim', 'diploidy', 'beamy', 'beamy', 'rumseller', 'battleground', 'pot-hunter', 'beamy', 'rumseller', 'diploidy', 'beamy', 'diploidy', 'beamy', 'beamy'] Bytes: 207.


# Challenges

Genetic algorithms do not usually appear to be the best way to solve problems. Especially in cases where analytical solutions exist and can be solved in a reasonable amount of time, there is no need for genetic methods. 

(Roulette wheel selection is also known as fitness-proportional selection because the probability of selection is proportional to the fitness measure of the chromosome.)

* Cannot guarantee discovery of solution in a predictable amount of time
* Appropriate only for situations where we can use a "good enough" solution
* Make configurable parameters to tweak

## Use Cases

* Complex optimization problems with no known analytical solution such as the Traveling Salesman Problem
* Mimic artwork using stochastic processes
* Scheduling problems where we need to find optimal routes between nodes in a network
* Machine learning hyperparameter optimization

One benefit of genetic algorithms is that they lend themselves easily to parallelization. As an example, each population can be simulated on a separate processor (crossover would require communication between populations). 